YÜKLEME VE VERİYİ KONTROL ETME İŞLEMLERİ

In [18]:
import pandas as pd
import nltk 
import numpy as np
import re

from nltk.stem import wordnet                                  # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer    # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer    # to perform tfidf
from nltk import pos_tag                                       # for parts of speech
from sklearn.metrics import pairwise_distances                 # to perfrom cosine similarity
from nltk import word_tokenize                                 # to create tokens
from nltk.corpus import stopwords                              # for stop words
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
data = pd.read_csv('/content/Mental_Health_FAQ.csv')
df = data.copy()

In [3]:
def check_df(df, head=5, tail=5):
    print(" SHAPE ".center(70, '*'))
    print('Observations -------> {}'.format(df.shape[0]))
    print('Features     -------> {}'.format(df.shape[1]))
    print(f"Shape of dataset: {(df.shape)}")
    print(" TYPES OF FEATURES ".center(70, '*'))
    print(df.dtypes,"\n")
    print(" HEAD ".center(70, '*'))
    print("\n",df.head(head),"\n")
    print(' TAIL '.center(70, '*'))
    print("\n",df.tail(tail),"\n")
    print(" INFO ".center(70, "*"))
    print("\n",df.info(),"\n")
    print(' DUPLICATE VALUE ANALYSIS '.center(70, '*'))
    print("\n",df.duplicated().sum(),"\n")
    print(" QUANTILES ".center(70, '*'))
    print("\n",df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T,"\n")
    print(" MISSING VALUES ".center(70, "*"))
    print(df.isnull().sum())

check_df(df)

******************************* SHAPE ********************************
Observations -------> 98
Features     -------> 3
Shape of dataset: (98, 3)
************************* TYPES OF FEATURES **************************
Question_ID     int64
Questions      object
Answers        object
dtype: object 

******************************** HEAD ********************************

    Question_ID                                          Questions  \
0      1590140        What does it mean to have a mental illness?   
1      2110618                    Who does mental illness affect?   
2      6361820                        What causes mental illness?   
3      9434130  What are some of the warning signs of mental i...   
4      7657263            Can people with mental illness recover?   

                                             Answers  
0  Mental illnesses are health conditions that di...  
1  It is estimated that mental illness affects 1 ...  
2  It is estimated that mental illness affects 1

<ipython-input-3-253c5fcf437c>:17: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print("\n",df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T,"\n")


Soru ve Cevap ları df1 e atayarak dataframe oluşturulması


In [4]:
df1 = df[["Questions", "Answers"]]
df1.head()

,Questions,Answers
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."


NLTK KULLANARAK VERİ TEMİZLEME

Bu kod parçası, NLTK kütüphanesindeki WordNet lemmatizer'ı kullanarak "absorbed" kelimesini fiil olarak lemmatize etmektedir.

İlk olarak, nltk.download('wordnet') komutunu kullanarak WordNet korpusunu indiriyoruz. Bu işlem, WordNet lemmatizer'ın çalışması için gereken veri kaynağını sağlar.
Ardından, from nltk.corpus import wordnet ifadesiyle wordnet modülünü içe aktarıyoruz. Bu modül, WordNet lemmatizer'ı ve diğer WordNet kaynaklarına erişim sağlar.
lemma = wordnet.WordNetLemmatizer() koduyla bir WordNetLemmatizer örneği oluşturuyoruz. Bu örnek, lemmatizasyon işlemini gerçekleştirmek için kullanılacaktır.
lemma.lemmatize('absorbed', pos='v') kodu, "absorbed" kelimesini fiil olarak lemmatize etmektedir. pos='v' parametresi, kelimenin fiil olduğunu belirtmektedir.
Son olarak, lemmatize edilmiş kelimeyi ekrana yazdırıyoruz.
Çıktı olarak, "absorbed" kelimesinin lemmatize edilmiş hali olan "absorb" kelimesi görüntülenir.

In [9]:
nltk.download('wordnet')                    
lemma = wordnet.WordNetLemmatizer()         
lemma.lemmatize('absorbed', pos = 'v')   

[nltk_data] Downloading package wordnet to /root/nltk_data...


'absorb'

İlk olarak, nltk.download('stopwords') komutunu kullanarak stopwords corpusunu indiriyoruz. Bu işlem, stopwords listesine erişmek için gereken veri kaynağını sağlar. Ardından, from nltk.corpus import stopwords ifadesiyle stopwords modülünü içe aktarıyoruz. Bu modül, stopwords listesine erişim sağlar. stopwords.words('english') koduyla İngilizce stopwords listesini alıyoruz. 'english' parametresi, İngilizce stopwords listesini belirtir. Elde edilen stopwords listesini ekrana yazdırıyoruz.

In [10]:
nltk.download('stopwords')            
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Bu fonksiyon, gelen metni küçük harfe dönüştürür, özel karakterleri kaldırır, kelimeleri ayırır, lemmatizasyon yapar ve stopwords listesinde yer alan kelimeleri çıkarır. Metnin daha temiz ve normalleştirilmiş bir versiyonunu döndürür.

text parametresi alır ve gelen metni küçük harfe dönüştürür.
re.sub(r'[^ a-z]','',text) ifadesi, metinden özel karakterleri kaldırır. Sadece harfleri içeren bir metin elde edilir.
nltk.word_tokenize(spl_char_text) ile metin, kelimelerine ayrılır ve bir kelime listesi oluşturulur.
wordnet.WordNetLemmatizer() ile WordNet lemmatizer'ı başlatılır.
pos_tag(tokens, tagset=None) ile kelimelerin parçalarına ayrılmış hali etiketlenir ve tags_list adında bir liste oluşturulur.
Bir döngü yardımıyla her kelimeye ve etiketine bakılır:
Kelimenin etiketi "V" ile başlıyorsa (pos_token.startswith('V')), fiil olarak kabul edilir ve pos_val değişkenine 'v' atanır.
Kelimenin etiketi "J" ile başlıyorsa (pos_token.startswith('J')), sıfat olarak kabul edilir ve pos_val değişkenine 'a' atanır.
Kelimenin etiketi "R" ile başlıyorsa (pos_token.startswith('R')), zarf olarak kabul edilir ve pos_val değişkenine 'r' atanır.
Yukarıdaki durumlar dışında, kelime isim olarak kabul edilir ve pos_val değişkenine 'n' atanır.
lema.lemmatize(token, pos_val) ile kelimenin lemmatize edilmiş hali elde edilir ve lema_token değişkenine atanır.
Eğer lemmatize edilmiş kelime stopwords listesinde (stop) yer alıyorsa, bu kelime lema_words listesine eklenir.
En son olarak, lema_words listesindeki kelimeler birleştirilerek dönüş metni olarak elde edilir.

In [19]:
def text_normalization(text):
    text = str(text).lower()                        # text to lower case
    spl_char_text = re.sub(r'[^ a-z]','',text)      # removing special characters
    tokens = nltk.word_tokenize(spl_char_text)      # word tokenizing
    lema = wordnet.WordNetLemmatizer()              # intializing lemmatization
    tags_list = pos_tag(tokens,tagset=None)         # parts of speech
    lema_words = []                                 # empty list 
    for token,pos_token in tags_list:               # lemmatize according to POS
        if pos_token.startswith('V'):               # Verb
            pos_val = 'v'
        elif pos_token.startswith('J'):             # Adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):             # Adverb
            pos_val = 'r'
        else:
            pos_val = 'n'                           # Noun
        lema_token = lema.lemmatize(token,pos_val)

        if lema_token in stop: 
          lema_words.append(lema_token)             # appending the lemmatized token into a list
    
    return " ".join(lema_words) 

Yukarıdaki fonksiyon çağrılarak normalizasyon işlemi yapılır

In [35]:
df1['lemmatized_questions'] = df1['Questions'].apply(text_normalization)   # clean text
df1.head(11)

,Questions,Answers,lemmatized_questions,similarity_bow
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,what do it to have a,0.316228
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...,who do,0.000000
2,What causes mental illness?,It is estimated that mental illness affects 1 ...,what,0.707107
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,what be some of the of,0.500000
4,Can people with mental illness recover?,"When healing from mental illness, early identi...",can with,0.000000
5,What should I do if I know someone who appears...,Although this website cannot substitute for pr...,what should i do if i who to have the of a,0.235702
6,How can I find a mental health professional fo...,Feeling comfortable with the professional you ...,how can i a for myself or my,0.000000
7,What treatment options are available?,Just as there are different types of medicatio...,what be,1.000000
8,"If I become involved in treatment, what do I n...",Since beginning treatment is a big step for in...,if i in what do i to,0.316228
9,What is the difference between mental health p...,There are many types of mental health professi...,what be the between,0.707107


CountVectorizer'ı başlatır ve lemmatized_questions sütunundaki metin verilerini vektörlere dönüştürmüş olursunuz. X değişkeni, her bir sorunun vektör temsilini içeren bir matris olacak.


In [22]:
cv = CountVectorizer() # intializing the count vectorizer
X = cv.fit_transform(df1['lemmatized_questions']).toarray()

Bu şekilde get_feature_names_out() yöntemini kullanarak CountVectorizer nesnesinden özellik isimlerini alır ve vektörleri bir DataFrame'e dönüştürürsünüz. df1_bow DataFrame'i, her bir sütunun bir kelimenin varlığını temsil eden bir BoW (Bag-of-Words) temsilini içerecektir.

In [27]:
# Özellik isimlerini alma
features = cv.get_feature_names_out()


# Vektörlerden DataFrame oluşturma
df1_bow = pd.DataFrame(X, columns=features)
df1_bow.head()






,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


ÖRNEK

Bu şekilde, Question değişkenindeki soruyu temizleyip lemmatize ediyoruz ve ardından transform() yöntemini kullanarak BoW temsilini elde ediyoruz. Question_bow değişkeni, verilen sorunun vektör temsilini içerecektir.

In [36]:
Question = 'What treatment options are available'    
Question_lemma = text_normalization(Question)                               # clean text
Question_bow = cv.transform([Question_lemma]).toarray()                     # applying bow

COUSINE BENZERLİĞİ

pairwise_distances fonksiyonunu kullanarak df1_bow ile Question_bow arasındaki kosinüs benzerliğini hesaplayabilirsiniz. cosine_value değişkeni, her soru için soruya olan benzerlik skorunu içerecektir.

In [37]:
# cosine similarity for the above question we considered.

cosine_value = 1- pairwise_distances(df1_bow, Question_bow, metric = 'cosine' )
(cosine_value)

array([[0.31622777],
       [0.        ],
       [0.70710678],
       [0.5       ],
       [0.        ],
       [0.23570226],
       [0.        ],
       [1.        ],
       [0.31622777],
       [0.70710678],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.40824829],
       [0.25      ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.70710678],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.26726124],
       [0.        ],
       [0.40824829],
       [0.70710678],
       [0.25      ],
       [0.        ],
       [0.28867513],
       [0.23570226],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.35355339],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

cosine_value olarak adlandırılan benzerlik skorunu df1 veri çerçevesine yeni bir sütun olarak ekler. Ardından, simiscores olarak adlandırılan bir DataFrame oluşturur, sadece 'Answers' ve 'similarity_bow' sütunlarını içerir. Bu DataFrame, alınan soru için yanıtların benzerlik değerini içerecektir.

In [38]:
df1['similarity_bow'] = cosine_value                                         # create cosine value as a new column
simiscores = pd.DataFrame(df1, columns=['Answers','similarity_bow'])         # taking similarity value of responses for the question we took
simiscores     

,Answers,similarity_bow
0,Mental illnesses are health conditions that di...,0.316228
1,It is estimated that mental illness affects 1 ...,0.000000
2,It is estimated that mental illness affects 1 ...,0.707107
3,Symptoms of mental health disorders vary depen...,0.500000
4,"When healing from mental illness, early identi...",0.000000
...,...,...
93,Sorting out if you are drinking too much can b...,0.000000
94,"Cannabis smoke, for example, contains cancer-c...",0.500000
95,You can't. But you can influence their capacit...,0.000000
96,Cannabidiol or CBD is a naturally occurring co...,0.632456


In [39]:
simscoresDescending = simiscores.sort_values(by = 'similarity_bow', ascending=False)          # sorting the values
simscoresDescending.head(15)

,Answers,similarity_bow
79,Cyclothymic disorder is a subtype of bipolar d...,1.000000
82,A personality disorder is a pattern of thought...,1.000000
57,MSP stands for Medical Services Plan. It’s a h...,1.000000
58,A referral means someone recommends you to ano...,1.000000
24,We all have mental health which is made up of ...,1.000000
89,Prodrome is a medical term for early signs or ...,1.000000
7,Just as there are different types of medicatio...,1.000000
85,Binge-eating disorder or BED is a type of eati...,1.000000
84,A personality disorder is a pattern of thought...,1.000000
83,A personality disorder is a pattern of thought...,1.000000


In [40]:
index_value = cosine_value.argmax()         # index number of highest value
index_value

7

In [41]:

df['Answers'].loc[index_value]              # The text at the above index becomes the response for the question

'Just as there are different types of medications for physical illness, different treatment options are available for individuals with mental illness. Treatment works differently for different people. It is important to find what works best for you or your child.'

Tf-Idf

 TfidfVectorizer sınıfını kullanarak TF-IDF (Term Frequency-Inverse Document Frequency) vektörlerini oluşturuyoruz. tfidf adında bir TfidfVectorizer nesnesi oluşturuyoruz ve daha sonra fit_transform yöntemini kullanarak lemmatized_questions sütununu dönüştürüyoruz.

In [43]:
Question1 = 'What treatment options are available'
tfidf = TfidfVectorizer()                                             # intializing tf-id 
x_tfidf = tfidf.fit_transform(df1['lemmatized_questions']).toarray()        # transforming the data into array

 önceki adımlarda tanımladığınız text_normalization işlevini kullanarak Question1 metnini normalize ediyoruz. Daha sonra, tfidf nesnesinin transform yöntemini kullanarak bu normalize edilmiş soruyu TF-IDF vektörüne dönüştürüyoruz.

In [44]:
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray()         # applying tf-idf

TF-IDF vektörlerini içeren x_tfidf matrisini kullanarak DataFrame oluşturuyoruz. tfidf nesnesinin get_feature_names_out() yöntemini kullanarak özellik isimlerini alıyoruz ve bu isimleri sütun isimleri olarak kullanarak DataFrame'i oluşturuyoruz.


In [45]:
# returns all the unique word from data with a score of that word

df1_tfidf = pd.DataFrame(x_tfidf,columns = tfidf.get_feature_names_out()) 
df1_tfidf.head()
     

,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.451966,0.0,0.0,0.299296,0.0,0.0,0.000000,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.877322,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.244128,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.211183,0.0,0.0,0.000000,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.408725,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.912658,0.0


df1_tfidf ve Question_tfidf arasındaki kosinüs benzerliğini hesaplıyoruz. pairwise_distances fonksiyonunu kullanarak, metric='cosine' parametresiyle kosinüs benzerliğini uyguluyoruz. Benzerlik skorunu cos değişkenine atıyoruz.

In [46]:

cos = 1-pairwise_distances(df1_tfidf,Question_tfidf,metric='cosine')                     # applying cosine similarity
cos

array([[0.1958092 ],
       [0.        ],
       [0.6542318 ],
       [0.32279463],
       [0.        ],
       [0.13411503],
       [0.        ],
       [1.        ],
       [0.19075897],
       [0.5997252 ],
       [0.        ],
       [0.19075897],
       [0.        ],
       [0.19684893],
       [0.17971191],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.57312936],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.19411811],
       [0.        ],
       [0.29317306],
       [0.6542318 ],
       [0.16840525],
       [0.        ],
       [0.17647424],
       [0.14237637],
       [0.        ],
       [0.30666465],
       [0.        ],
       [0.27868552],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

Aşağıdaki kod, cos değişkenindeki benzerlik skorlarını df1 veri çerçevesine similarity_tfidf adlı yeni bir sütun olarak ekler:

Daha sonra, df1 veri çerçevesinden sadece 'Answers' ve 'similarity_tfidf' sütunlarını içeren df1_simi_tfidf adlı yeni bir veri çerçevesi oluşturulur:

Bu şekilde, df1_simi_tfidf veri çerçevesi, alınan soru için yanıtların benzerlik değerlerini içeren bir veri çerçevesi olarak elde edilir.

In [47]:
df1['similarity_tfidf'] = cos                                                    # creating a new column 
df1_simi_tfidf = pd.DataFrame(df1, columns=['Answers','similarity_tfidf'])        # taking similarity value of responses for the question we took
df1_simi_tfidf

,Answers,similarity_tfidf
0,Mental illnesses are health conditions that di...,0.195809
1,It is estimated that mental illness affects 1 ...,0.000000
2,It is estimated that mental illness affects 1 ...,0.654232
3,Symptoms of mental health disorders vary depen...,0.322795
4,"When healing from mental illness, early identi...",0.000000
...,...,...
93,Sorting out if you are drinking too much can b...,0.000000
94,"Cannabis smoke, for example, contains cancer-c...",0.372840
95,You can't. But you can influence their capacit...,0.000000
96,Cannabidiol or CBD is a naturally occurring co...,0.514957


In [49]:
df1_simi_tfidf_sort = df1_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False)            # sorting the values
df1_simi_tfidf_sort.head(15)

,Answers,similarity_tfidf
82,A personality disorder is a pattern of thought...,1.000000
79,Cyclothymic disorder is a subtype of bipolar d...,1.000000
57,MSP stands for Medical Services Plan. It’s a h...,1.000000
89,Prodrome is a medical term for early signs or ...,1.000000
7,Just as there are different types of medicatio...,1.000000
24,We all have mental health which is made up of ...,1.000000
85,Binge-eating disorder or BED is a type of eati...,1.000000
84,A personality disorder is a pattern of thought...,1.000000
83,A personality disorder is a pattern of thought...,1.000000
58,A referral means someone recommends you to ano...,1.000000


In [50]:

index_value1 = cos.argmax()                                                   # returns the index number of highest value
index_value1

7

In [51]:
df1['Answers'].loc[index_value1]    # returns the text at that index

'Just as there are different types of medications for physical illness, different treatment options are available for individuals with mental illness. Treatment works differently for different people. It is important to find what works best for you or your child.'

CHATBOTU TEST ETME

Aşağıdaki kod, bir metin girdisine dayalı olarak benzerlik tabanlı bir sohbet işlevi chat_bow tanımlar. İşlev, aşağıdaki adımları izler:

Girdi metni üzerinde metin normalizasyonu yapmak için text_normalization fonksiyonunu çağırır.

Bow (bag-of-words) dönüşümünü uygulamak için cv.transform işlevini kullanır.

Cosine benzerliği hesaplamak için pairwise_distances fonksiyonunu kullanır.

Elde edilen benzerlik değerlerinden en yüksek değeri alarak, en yakın yanıtın indis değerini bulur.

df1 veri çerçevesinde bu indis değerine karşılık gelen yanıtı döndürür.

Bu şekilde, chat_bow işlevini kullanarak bir girdi metni verildiğinde, en yakın yanıtı elde edebilirsiniz.

In [52]:
# defining a function that returns response to query using bow

def chat_bow(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    bow = cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df1_bow,bow, metric = 'cosine' )
    index_value = cosine_value.argmax() # getting index value 
    return df1['Answers'].loc[index_value]

Aşağıdaki kod, bir metin girdisine dayalı olarak TF-IDF tabanlı bir sohbet işlevi chat_tfidf tanımlar. İşlev, aşağıdaki adımları izler:

Girdi metni üzerinde metin normalizasyonu yapmak için text_normalization fonksiyonunu çağırır.

TF-IDF dönüşümünü uygulamak için tfidf.transform işlevini kullanır.
Cosine benzerliği hesaplamak için pairwise_distances fonksiyonunu kullanır.

Elde edilen benzerlik değerlerinden en yüksek değeri alarak, en yakın yanıtın indis değerini bulur.

df1 veri çerçevesinde bu indis değerine karşılık gelen yanıtı döndürür.

Bu şekilde, chat_tfidf işlevini kullanarak bir girdi metni verildiğinde, en yakın yanıtı elde edebilirsiniz.

In [56]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    tf = tfidf.transform([lemma]).toarray() # applying tf-idf
    cos = 1-pairwise_distances(df1_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value = cos.argmax() # getting index value 
    return df1['Answers'].loc[index_value]
     

BOW VE TF IDF KULLANILARAK CHATBOT LARIN KARŞILAŞTIRILMASI

In [55]:
chat_bow('Who does mental illness affect')

'It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness. Mental illness does not discriminate; it can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background.\nAlthough mental illness can affect anyone, certain conditions may be more common in different populations. For instance, eating disorders tend to occur more often in females, while disorders such as attention deficit/hyperactivity disorder is more prevalent in children.\nAdditionally, all ages are susceptible, but the young and the old are especially vulnerable. Mental illnesses usually strike individuals in the prime of their lives, with 75 percent of mental health conditions developing by the age of 24. This makes identification and treatment of mental disorders particularly difficult, because the normal personality and behavioral changes of adolescence may mask symptoms of a mental health condit

In [57]:
chat_tfidf('Who does mental illness affect')

'It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness. Mental illness does not discriminate; it can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background.\nAlthough mental illness can affect anyone, certain conditions may be more common in different populations. For instance, eating disorders tend to occur more often in females, while disorders such as attention deficit/hyperactivity disorder is more prevalent in children.\nAdditionally, all ages are susceptible, but the young and the old are especially vulnerable. Mental illnesses usually strike individuals in the prime of their lives, with 75 percent of mental health conditions developing by the age of 24. This makes identification and treatment of mental disorders particularly difficult, because the normal personality and behavioral changes of adolescence may mask symptoms of a mental health condit

In [58]:
chat_bow('Can people with mental illness recover')

'When healing from mental illness, early identification and treatment are of vital importance. Based on the nature of the illness, there are a range of effective treatments available. For any type of treatment, it is essential that the person affected is proactive and fully engaged in their own recovery process.\nMany people with mental illnesses who are diagnosed and treated respond well, although some might experience a return of symptoms. Even in such cases, with careful monitoring and management of the disorder, it is still quite possible to live a fulfilled and productive life.'

In [59]:
chat_tfidf('Can people with mental illness recover')

'When healing from mental illness, early identification and treatment are of vital importance. Based on the nature of the illness, there are a range of effective treatments available. For any type of treatment, it is essential that the person affected is proactive and fully engaged in their own recovery process.\nMany people with mental illnesses who are diagnosed and treated respond well, although some might experience a return of symptoms. Even in such cases, with careful monitoring and management of the disorder, it is still quite possible to live a fulfilled and productive life.'

In [65]:
chat_bow('Where can I go to find therapy')

'Different kinds of therapy are more effective based on the nature of the mental health condition and/or symptoms and the person who has them (for example, children will benefit from a therapist who specializes in children’s mental health). However, there are several different types of treatment and therapy that can help.'

In [66]:
chat_tfidf('Where can I go to find therapy')

'Different kinds of therapy are more effective based on the nature of the mental health condition and/or symptoms and the person who has them (for example, children will benefit from a therapist who specializes in children’s mental health). However, there are several different types of treatment and therapy that can help.'

Sonuç olarak BOW ve TF IDF yöntemleriyle CHATBOT oluşturma yöntemlerini incelediğimizde ikisinin de aynı sonuç verdiğini görmekteyiz.